### Import libraries

In [ ]:
# import libraries

# excel (.xlsx) file
from openpyxl import load_workbook

# json file
import json

# directory 사용
import os

# 파일/폴더 이동, 제거
import shutil

# numpy
import numpy as np



### Excel files

In [ ]:
# open excel file
wb_file_name = "excel_file_name"
wb = load_workbook(wb_file_name + ".xlsx" , data_only=True )
ws = wb.active

# print excel file (엑셀 내용 출력)
row_num = 1 # excel row number
for row in ws.values:
    print(row_num, end="\t")
    for i in range(len(row)):
        print(row[i], end="\t")
    print()
    row_num += 1

In [ ]:
# 엑셀의 2번째 열만 출력
for row in ws.values:
    print(row[1])

In [ ]:
# edit excel file
# 엑셀 파일 수정 - 2번째 열의 공백 제거
row_num = 1
for row in ws.values:
    if row[1] == " PID / NAME MISMATCH ": # " PID / NAME MISMATCH " 진단명의 맨앞 공백 1칸을 제거함
        print("1")
        ws.cell(row=row_num, column=2, value=row[1][1:])
    elif row[1] != "diagnosis": # 첫번째 행은 머리글 - "diagnosis" 라는 머리글 제외하고 진단명의 맨앞 공백 2칸을 제거함
        print("2")
        ws.cell(row=row_num, column=2, value=row[1][2:])
    row_num += 1

In [ ]:
# edit excel file
# 엑셀 파일 수정 - 파일 이름에 진단명 넣기

row_num = 1 # excel row number
for rf in os.listdir(hsecg_data_dir):
    print(rf) # rs_10001_01
    #print("row=", row_num, "column=", 1, "value=", rf)
    #ws.cell(row=row_num, column=1, value=rf)
    sub_dir = os.path.join(hsecg_data_dir, rf)
    print("sub_dir", sub_dir) # ./1_yoingin_respiratory_sound/rs_10001_01
    for file in os.listdir(sub_dir):
        print("row=", row_num, "column=", 1, "value=", file)
        ws.cell(row=row_num, column=1, value=file)
        row_num += 1
    print()

In [ ]:
# edit excel file
# 엑셀 파일 수정 - get_list_of_respiratory_sound_files
line_num = 1
for i in os.listdir():
    print(i)
    if i == "yongin_ecg_diagnosis.xlsx" or i == ".ipynb_checkpoints" or i == "20221110.ipynb": # ecg 파일이 아닐 경우 스킵합니다!
        print("skip!\n")
    else:
        j = i[:-4] # ".pdf" 지우기
        print(j)
        row_num = 1
        for row in ws.values:
            if str(row[0]) == j: # pid가 매칭될 경우
                if row[1] != "PID / NAME MISMATCH ": # 진단명이 "PID / NAME MISMATCH "일 경우 빼고!
                    os.rename(i, j + "_" + row[1] + ".pdf")
                    print(line_num, row_num, i, "완료")
            row_num += 1
    line_num += 1

In [ ]:
# save excel file (수정한 것을 적용)
wb.save(wb_file_name + "_completed.xlsx")

### Text files

In [ ]:
def replace_in_file(file_path, old_str, new_str):
    # 파일 읽어들이기
    fr = open(file_path, 'r')
    lines = fr.readlines()
    fr.close()
    
    # old_str -> new_str 치환
    fw = open(file_path, 'w')
    for line in lines:
        fw.write(line.replace(old_str, new_str))
    fw.close()

In [ ]:
# open text file
f = open("text_file_name.txt", 'r')
lines = f.readlines()

In [ ]:
# close text file
f.close()

### Json files

In [ ]:
# txt 파일명에서 볼 수 있는 진단된 label을 (ex. 0 (=normal))
# 진단명에 맞게 json의 label에 넣기
# 단, json의 label이 99일 때만!

respiratory_dir = "./respiratory_sound/"

for rf in os.listdir(respiratory_dir):
    sub_dir = os.path.join(respiratory_dir, rf)
    with open(sub_dir + "/" + rf + ".json", "r+", encoding="utf8") as f:
        json_data = json.load(f)
        print("before=", json_data)
        json_data_rf = json_data["respiratory_files"] # unpacking json_data and reading labels in json_data in json file
        #print(json_data_rf)
        for i, data in enumerate(json_data_rf):
            if (data[0]["label"] == 99):
                for file in os.listdir(sub_dir):
                    #print(file[-3:])
                    #print(file[:-6])
                    if file[-3:] == "txt" and file[:16] == data[0]["id"]:
                        txt_label = file.rsplit('_')[-1][:-4]
                        #print(txt_label)
                        #print("before=", json_data["respiratory_files"][i][0]["label"])
                        json_data["respiratory_files"][i][0]["label"] = int(txt_label)
                        #print("after=", json_data["respiratory_files"][i][0]["label"])

    print("after=", json_data)
    print()
    
    after_file = open(sub_dir + "/" + rf + ".json", "w", encoding="utf8")
    json.dump(json_data, after_file, indent=4)
    after_file.close()

### 파일 개수 확인하기

In [ ]:
# 각 PID 별 호흡음 파일이 4개 이상 있는지 확인

hsecg_data_dir = "./" + "4_smartsound_respiratory_sound" + "/"

for rf in os.listdir(hsecg_data_dir):
    sub_dir = os.path.join(hsecg_data_dir, rf)
    # print("sub_dir", sub_dir) # ./1_yoingin_respiratory_sound/rs_10001_01
    wav_file_num = 0
    if len(os.listdir(sub_dir)) == 0:
        print("no file exists", rf) # 폴더에 파일이 아예 없다면...
    for file in os.listdir(sub_dir):
        #print(file[-3:]) # wav
        if file[-3:] == "wav":
            wav_file_num += 1
        else:
            print(file) # 파일 확장명이 wav가 아니면 파일명 출력
        if file == os.listdir(sub_dir)[-1] and wav_file_num < 4:
            print("less than 4 wav files", rf) # 호흡음 파일이 4개 미만이라면 출력
    if (wav_file_num == 12): # 호흡음 파일이 12개라면 출력
        print(rf)

### 폴더 지우기

In [ ]:
# 날짜있는 폴더가 2개 있는 것 중 아래 폴더 지우기

root_dir = "./RespiratorySound/"  
files = os.listdir(root_dir)   
for file in files:
    sub_dir = os.path.join(root_dir, file) # sub_dir = root_dir + file
    #print(sub_dir) # ./RespiratorySound/1427458
    sub_files = os.listdir(sub_dir)
    for sub_file in sub_files:
        sub_2_dir = os.path.join(sub_dir, sub_file) # sub_2_dir = sub_dir + sub_file
        #print(sub_2_dir) # ./RespiratorySound/1427458\Lung Sound (폐음)
        sub_2_files = os.listdir(sub_2_dir)
        #print(sub_2_files)
        if len(sub_2_files) != 1:
            #print(sub_2_files)
            #print(sub_2_dir)
            i = 1
            for sub_2_file in sub_2_files:
                path = os.path.join(sub_2_dir, sub_2_file)
                print(path) # ./RespiratorySound/1427458\Lung Sound (폐음)\2022-07-28-140453
                if i == 2: # 두 번째 폴더 지우기
                    print("remove", path) # remove ./RespiratorySound/1427458\Lung Sound (폐음)\2022-07-28-141444
                    shutil.rmtree(path) # 폴더와 그 안의 파일 모두 삭제
                i += 1
            print()